In [103]:
import pickle
#Load Dictionary
with open('transcripts_timestamps.pkl', 'rb') as infile:
    transcripts = pickle.load(infile)

In [1]:
# transcripts

In [105]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "Title": {"type": "text"},
            "Text": {"type": "text"},
            "TimeStamp": {"type": "float"}
        }
    }
}

In [106]:
from tqdm.auto import tqdm

In [120]:
from elasticsearch import Elasticsearch
index_name = 'transcripts'
es_client = Elasticsearch('http://localhost:9200')
for doc in tqdm(transcripts):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/4200 [00:00<?, ?it/s]

In [134]:
#Question3 Searching - 
def retrieve_documents_question(query, index_name="transcripts", max_results=28):
    es = Elasticsearch("http://localhost:9200")
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["Title", "Text^4"],
                        "type": "most_fields"
                    }
                },
            }
        }
    }
    
    try:
        response = es.search(index=index_name, body=search_query)
        result_docs = [hit['_source'] for hit in response['hits']['hits']]
        return result_docs
    except Exception as e:
        print(f"Error retrieving documents: {e}")
        return []

In [2]:
# retrieve_documents_question('Elasticsearch?')

In [136]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [137]:
def make_prompt(question):
    context = retrieve_documents_question(question)
    
    prompt = f""" 
    You are a very helpful teaching assistant for an LLM Course. Given the student's question below, use the context that follows to direct them to the correct video they are looking for.
    But also give a short 5 sentence summary of what the entire video covers. Make sure that you only point students to videos which are in the CONTEXT section below. Try to find where records
    are clustered. So if someone is asking a question and a lot of the results are in one video based on Title and the timestamp are around the same time more or less, instruct
    them to check that video title around where most of the timestamps are.
    
    QUESTION: In what video is {question}
    
    CONTEXT: {context}
    """
    
    return prompt.strip()

In [138]:
prompt = make_prompt('In what videos do we start talking about vector search?')

In [139]:
response = client.chat.completions.create(
    model='phi3',
    messages=[{"role": "user", "content": prompt}]
)

In [140]:
print(response.choices[0].message.content)

 The videos where the discussion about Vector Search is happening are:

1. Title: 'llm zoomcamp 3.1 - introduction to vector search'  
Timestamps: 515.68, 429.44, 471.08, 403.44   
Summary: This video introduces the concept of Vector Search and emphasizes on how we can store it efficiently in a vector format. The entire course covers various aspects such as understanding what an Vector is, why it's extremely important to learn about vector search concepts and the different approaches for storing these vectors effectively.
